In [ ]:
def getWhitePixels(image):
    numWhite = 0
    for i in im_bw:
        for j in i:
            if (j):
                numWhite += 1
    return numWhite

In [ ]:
pixels = []
for i in ic:
    im = i
    im_float = (im.astype(float) - im.min()) / (im.max() - im.min())
    im_edge = skimage.feature.canny(im_float, 1.4)
    im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)
    # Close the edge image
    selem = skimage.morphology.disk(2)
    im_edge_closed = skimage.morphology.binary_closing(im_edge, selem)

    # Fill these holes
    im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge_closed)
    im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, 2.0)
    # 3x3 square structuring element
    selem = skimage.morphology.square(3)
    
    # Do max filter and min filter
    im_LoG_max = scipy.ndimage.filters.maximum_filter(im_LoG, footprint=selem)
    im_LoG_min = scipy.ndimage.filters.minimum_filter(im_LoG, footprint=selem)

    # Image of zero-crossings
    im_edge = ((im_LoG >= 0) & (im_LoG_min < 0)) \
        | ((im_LoG <= 0) & (im_LoG_max > 0))
    im_edge = zero_crossing_filter(im_LoG, 0.001)
    im_edge = skimage.morphology.skeletonize(im_edge)
    # Fill holes
    im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

    # Remove small objectes that are not bacteria
    im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=100)

    im_bw = skimage.segmentation.clear_border(im_bw, buffer_size=5)
    pixels.append(getWhitePixels(im_bw))

